This notebook will compare model architecture, hyperparameters, inputs, and training/validation error

In [ ]:
import xarray as xr
import numpy as np
from src.crossval import run_crossval

In [ ]:
input_dir = "/glade/work/milesep/convective_outlook_ml"
target_dir = "data/processed_data"
stats_dir = "data/processed_data"

In [ ]:
# compare all models, save training and validation curves along with corresponding model name, data level, model/training hyperparameters
model_names = ["cnn3d"]
levels = ["full"]
# Don't do linear regression with full dataset--too many parameters
for name, level in zip(model_names, levels):
    print(name, level)
    inputs = xr.open_zarr(f"{input_dir}/train_inputs_{level}.zarr")
    targets = xr.open_dataset(f"{target_dir}/train_targets.nc")
    stats = xr.open_dataset(f"{stats_dir}/daily_input_stats_{level}.nc")
    scores = run_crossval(inputs, targets, name, epochs = 40, level = level, restart = False)
    print(f"{name}: {np.mean(scores):.3f} ± {np.std(scores):.3f}")